In [13]:
import os

import librosa
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from matplotlib import pyplot as plt
from pydub import AudioSegment
from scipy import signal
from sklearn import preprocessing

In [14]:
pathAudio = 'C:/Users/Admin/OneDrive/Dokumente/Studium/Technology Lab/Technology Lab Team 4/Techno Titel/train-stochastic/samples/'
files = librosa.util.find_files(pathAudio, ext=['mp3'])
files = sorted(files, key=lambda x: int(x.split('_')[-1].split(".")[0]))

In [15]:
def normalize_sample(sample):
    array_of_samples = sample.get_array_of_samples()
    np_arr = np.array(array_of_samples)
    np_arr = np_arr.reshape(1, -1)
    return preprocessing.normalize(np_arr)

samples = []
samples_sec_canal = []
for file_name in files:
    sample = AudioSegment.from_mp3(file_name)
    mono_samples = sample.split_to_mono()
    normalized_sample = normalize_sample(mono_samples[0])[0]
    resampled_sample = signal.resample(normalized_sample, 1000)
    samples.append(resampled_sample)
    normalized_sample_sec_canal = normalize_sample(mono_samples[1])[0]
    resampled_sample_sec_canal = signal.resample(normalized_sample_sec_canal, 1000)
    samples_sec_canal.append(resampled_sample_sec_canal)

In [16]:
X_train = []
Y_train = []
for i in range(len(samples)):
    Y_train.append(i)
    if i < len(samples) - 1:
        x = list(samples[i])
        y = list(samples[i + 1])
        z = x + y
        X_train.append(z)
    else:
        x = list(samples[i])
        y = list(samples[0])
        z = x + y
        X_train.append(z)

In [17]:
from numpy import random

X_train_false = []


def get_strange_end_part(samples, current_index, max_len):
    x = random.randint(max_len)
    if x != current_index:
        return samples[x]
    else:
        return get_strange_end_part(samples, current_index, max_len)


for i in range(len(samples)):
    first_half = samples[i]
    last_half = get_strange_end_part(samples, i, len(samples))
    new_sample = list(first_half) + list(last_half)
    X_train_false.append(new_sample)

In [18]:
X_train = np.array(X_train)
X_train = X_train.reshape(-1, 2000)

In [19]:
from keras.utils.np_utils  import to_categorical

Y_train = to_categorical(Y_train)

In [20]:
Y_train = np.array(Y_train)
Y_train.shape

(151, 151)

In [21]:
X_train = X_train.reshape(-1, 2000)
X_train.shape

(151, 2000)

In [22]:
from keras.layers import Conv1D, Softmax, LSTM, MaxPool1D, Reshape, Flatten

model = Sequential()
model.add(Conv1D(filters=10, kernel_size=1, input_shape=(2000, 1)))
model.add(LSTM(20, return_sequences=True))
model.add(Dense(100))
model.add(Flatten())
model.add(Dense(151, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 2000, 10)          20        
                                                                 
 lstm_1 (LSTM)               (None, 2000, 20)          2480      
                                                                 
 dense_2 (Dense)             (None, 2000, 100)         2100      
                                                                 
 flatten_1 (Flatten)         (None, 200000)            0         
                                                                 
 dense_3 (Dense)             (None, 151)               30200151  
                                                                 
Total params: 30,204,751
Trainable params: 30,204,751
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train, Y_train, epochs=50)

Epoch 1/100
5/5 [==============================] - 4s 554ms/step - loss: 6.2149 - accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 3s 551ms/step - loss: 5.1626 - accuracy: 0.0331
Epoch 3/100
5/5 [==============================] - 3s 550ms/step - loss: 5.0407 - accuracy: 0.0199
Epoch 4/100
5/5 [==============================] - 3s 548ms/step - loss: 4.9909 - accuracy: 0.0199
Epoch 5/100
5/5 [==============================] - 3s 551ms/step - loss: 5.0420 - accuracy: 0.0066
Epoch 6/100
5/5 [==============================] - 3s 584ms/step - loss: 4.9541 - accuracy: 0.0464
Epoch 7/100
5/5 [==============================] - 3s 549ms/step - loss: 5.0038 - accuracy: 0.0530
Epoch 8/100
5/5 [==============================] - 3s 546ms/step - loss: 4.8440 - accuracy: 0.0265
Epoch 9/100
5/5 [==============================] - 3s 549ms/step - loss: 4.7749 - accuracy: 0.0464
Epoch 10/100
5/5 [==============================] - 3s 550ms/step - loss: 4.6685 - accuracy: 0.1060
Epoch

In [25]:
example_x_train = X_train[5]
example_x_train = example_x_train.reshape(1, 2000)
model.predict(example_x_train)

array([[0.1155197 , 0.09499806, 0.10819641, 0.24734515, 0.08010966,
        0.99999976, 0.03235859, 0.09354329, 0.7644672 , 0.43629235,
        0.87785304, 0.45638436, 0.18086553, 0.21477497, 0.17617598,
        0.46413058, 0.84987545, 0.2363402 , 0.31426448, 0.32180023,
        0.22280705, 0.3568207 , 0.12246639, 0.08876893, 0.09041423,
        0.1022242 , 0.1179395 , 0.04676923, 0.0652498 , 0.1524972 ,
        0.20655802, 0.08659616, 0.09586105, 0.08443609, 0.19441497,
        0.120915  , 0.134485  , 0.05067721, 0.08827373, 0.10430866,
        0.08999884, 0.21596971, 0.02025309, 0.17312726, 0.0803861 ,
        0.10938597, 0.04605448, 0.07781839, 0.19186291, 0.13659441,
        0.13092047, 0.05411488, 0.10392627, 0.08109275, 0.12193373,
        0.14410487, 0.17297137, 0.13988599, 0.04560044, 0.12381893,
        0.09330493, 0.17821988, 0.09398633, 0.53253984, 0.12086371,
        0.06125689, 0.08759531, 0.08359504, 0.09267509, 0.11143452,
        0.11194801, 0.43615732, 0.35498935, 0.05

In [26]:
example_x_train = X_train[7]
example_x_train = example_x_train.reshape(1, 2000)
model.predict(example_x_train)

array([[9.72260535e-02, 8.99455547e-02, 8.38040709e-02, 1.42036051e-01,
        9.65026021e-02, 1.17318153e-01, 9.32859480e-02, 1.00000000e+00,
        6.61250412e-01, 4.39455390e-01, 8.02544832e-01, 6.25939310e-01,
        3.06759775e-01, 1.72424048e-01, 7.39420057e-06, 4.69055712e-01,
        7.62838066e-01, 3.98360431e-01, 1.35889500e-01, 3.56560946e-01,
        1.68748915e-01, 4.56203401e-01, 1.04245871e-01, 8.49641562e-02,
        9.12809670e-02, 1.00424677e-01, 1.36928111e-01, 7.12146461e-02,
        8.66831243e-02, 6.01220727e-02, 5.86901903e-02, 9.92038846e-02,
        1.08283550e-01, 8.72550607e-02, 3.75804901e-02, 1.00272179e-01,
        7.34792352e-02, 9.79675055e-02, 1.97563708e-01, 1.41380548e-01,
        6.82972372e-02, 5.14945686e-02, 9.64626968e-02, 6.94693625e-02,
        5.65363765e-02, 7.68977404e-02, 5.87127209e-02, 6.01515174e-02,
        4.62696850e-02, 1.17538452e-01, 9.16757286e-02, 1.21439368e-01,
        1.27818912e-01, 1.41320765e-01, 9.35410857e-02, 7.882902

In [27]:
example_x_train = X_train_false[0]
example_x_train = np.array(example_x_train)
example_x_train = example_x_train.reshape(1, 2000)
model.predict(example_x_train)

array([[6.07429981e-01, 6.35533035e-02, 2.97198594e-02, 1.46061182e-03,
        4.82842624e-02, 8.62190425e-02, 8.64733160e-02, 2.70354748e-02,
        5.18209934e-01, 1.81520343e-01, 6.93426073e-01, 4.35563743e-01,
        9.40398276e-02, 4.39375043e-02, 7.20770061e-02, 1.89518780e-01,
        6.88767672e-01, 2.89665520e-01, 4.44339216e-01, 1.85474575e-01,
        1.72162324e-01, 6.09632432e-01, 1.30386233e-01, 5.41687012e-02,
        2.45497316e-01, 6.77424550e-01, 4.64788079e-02, 7.08445907e-03,
        1.63012743e-03, 6.87070370e-01, 5.83828688e-01, 2.07768083e-02,
        2.19884515e-03, 9.19938087e-04, 8.47866237e-02, 1.66740417e-02,
        4.02021408e-03, 3.47720921e-01, 2.91358829e-02, 9.78887081e-04,
        1.32051110e-03, 8.96617591e-01, 2.31447816e-02, 1.03093058e-01,
        3.87389362e-02, 4.47377443e-01, 3.82290781e-02, 2.07088709e-01,
        2.47540414e-01, 4.94676858e-01, 1.15307182e-01, 1.50100589e-01,
        9.99886990e-01, 9.40875113e-02, 2.84490585e-02, 7.414929